<a href="https://colab.research.google.com/github/RamzNN/RamzNN/blob/main/Filter_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1p3SDoe58aGKtt-4NnKz38pXGf2lUOXje

Downloading...
From: https://drive.google.com/uc?id=1p3SDoe58aGKtt-4NnKz38pXGf2lUOXje
To: /content/выгрузка_вк.xlsx
100% 17.9M/17.9M [00:00<00:00, 44.0MB/s]


In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_excel('/content/выгрузка_вк.xlsx')

In [4]:
df.tail(2)

,group_id,name,description,Subscribers Count,relevance,Threat assesment status,city,country,tags,subcultures
54281,56146201,Великі Вуйки/Останній москаль,NaN,23295,True,destructive,Киев,Украина,Москаль,Политический экстремизм
54282,55284725,Команда Навального,NaN,205372,True,destructive,NaN,NaN,Навальный,Политический экстремизм


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54283 entries, 0 to 54282
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   group_id                 53583 non-null  object
 1   name                     54280 non-null  object
 2   description              40436 non-null  object
 3   Subscribers Count        53887 non-null  object
 4   relevance                54283 non-null  bool  
 5   Threat assesment status  53968 non-null  object
 6   city                     51682 non-null  object
 7   country                  52236 non-null  object
 8   tags                     54283 non-null  object
 9   subcultures              54283 non-null  object
dtypes: bool(1), object(9)
memory usage: 3.8+ MB


In [6]:
# Проверка пропусков
print("\nКоличество пропусков по столбцам:")
print(df.isnull().sum())

# Распределение целевой переменной 'relevance'
print("\nРаспределение 'relevance':")
print(df['relevance'].value_counts())


Количество пропусков по столбцам:
group_id                     700
name                           3
description                13847
Subscribers Count            396
relevance                      0
Threat assesment status      315
city                        2601
country                     2047
tags                           0
subcultures                    0
dtype: int64

Распределение 'relevance':
relevance
False    45142
True      9141
Name: count, dtype: int64


In [7]:
# Если name пустое — заполним пустой строкой
df['name'] = df['name'].fillna('')
# Если description пустое — заполним пустой строкой
df['description'] = df['description'].fillna('')

In [8]:
# Проверка пропусков
print("\nКоличество пропусков по столбцам:")
print(df.isnull().sum())

# Распределение целевой переменной 'relevance'
print("\nРаспределение 'relevance':")
print(df['relevance'].value_counts())


Количество пропусков по столбцам:
group_id                    700
name                          0
description                   0
Subscribers Count           396
relevance                     0
Threat assesment status     315
city                       2601
country                    2047
tags                          0
subcultures                   0
dtype: int64

Распределение 'relevance':
relevance
False    45142
True      9141
Name: count, dtype: int64


In [9]:
# Объединение названия и описания
df['text'] = df['name'].astype(str) + ' ' + df['description'].astype(str)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54283 entries, 0 to 54282
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   group_id                 53583 non-null  object
 1   name                     54283 non-null  object
 2   description              54283 non-null  object
 3   Subscribers Count        53887 non-null  object
 4   relevance                54283 non-null  bool  
 5   Threat assesment status  53968 non-null  object
 6   city                     51682 non-null  object
 7   country                  52236 non-null  object
 8   tags                     54283 non-null  object
 9   subcultures              54283 non-null  object
 10  text                     54283 non-null  object
dtypes: bool(1), object(10)
memory usage: 4.2+ MB


In [11]:
df['relevance'].value_counts()

,count
relevance,
False,45142
True,9141


In [12]:
def clean_text(text):
      text = text.lower()
      text = re.sub(r'\s+', ' ', text).strip()
      return text

df['text'] = df['text'].apply(clean_text)

In [13]:
df['text_length'] = df['text'].apply(lambda x: len(x.split()))
df['text_length'].describe()

,text_length
count,54283.000000
mean,65.324061
std,112.336866
min,1.000000
25%,4.000000
50%,18.000000
75%,71.000000
max,734.000000


In [14]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['relevance'])
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['relevance'])

### Oversampling

Как именно добавляем 7k?

- Мы берём случайную выборку (с replace=True) из уже имеющихся примеров класса True.
- Приклеиваем эти сэмплы к train_df.
- Перемешиваем, чтобы избежать кластеризации.

Что это даст?

- В train_df станет больше примеров True, что может поднять Recall(True), но потенциально может слегка снизить Precision(True).
- Лучше экспериментально проверить, насколько именно 7k дополнительно помогает, может иметь смысл подбирать число итеративно.

In [15]:
import pandas as pd

# 1) Разделяем train_df на True/False
train_df_true = train_df[train_df['relevance'] == 1]
train_df_false = train_df[train_df['relevance'] == 0]

print("Исходный train: ", train_df.shape)
print("   True:", len(train_df_true), "  False:", len(train_df_false))

# 2) Делаем oversampling: берем 6k примеров True
#    sample(..., replace=True) => выборка с возвращением
df_true_additional = train_df_true.sample(n=6000, replace=True, random_state=42)

# 3) Соединяем их с исходным train
train_df_oversampled = pd.concat([train_df, df_true_additional], axis=0)

# 4) Перемешиваем строки
train_df_oversampled = train_df_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

print("\nПосле oversampling: ", train_df_oversampled.shape)
print("   True:", sum(train_df_oversampled['relevance'] == 1),
      "  False:", sum(train_df_oversampled['relevance'] == 0))


Исходный train:  (43968, 12)
   True: 7404   False: 36564

После oversampling:  (49968, 12)
   True: 13404   False: 36564


In [16]:
train_df = train_df_oversampled

### Установка нужных библиотек

In [17]:
!pip install --upgrade pip
!pip install transformers accelerate evaluate torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 

In [18]:
import pandas as pd
import numpy as np
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

# Убедимся, что у нас есть CUDA (если доступно)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Предположим, вы уже прочитали df, train_df, val_df, test_df
# как мы делали ранее

Using device: cuda


In [19]:
train_df['relevance'] = train_df['relevance'].astype(int)
val_df['relevance'] = val_df['relevance'].astype(int)
test_df['relevance'] = test_df['relevance'].astype(int)

In [20]:
model_name = "ai-forever/ruBert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2  # бинарная классификация
)
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/716M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

### Подготовка данных для Trainer

In [21]:
MAX_LEN = 256  # Так как средняя длина ~65 слов, но max ~734, выберем 512

def tokenize_function(examples):
    # examples – это dict с ключами "text" и "relevance"
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding="max_length",  # или "longest", но max_length=512 безопаснее для batched training
    )

In [22]:
# Для Trainer нужно, чтобы в словаре для каждой строчки были хотя бы поля: 'text' и 'labels' (или 'label').

train_data = {
    "text": train_df["text"].tolist(),
    "labels": train_df["relevance"].tolist()
}

val_data = {
    "text": val_df["text"].tolist(),
    "labels": val_df["relevance"].tolist()
}

test_data = {
    "text": test_df["text"].tolist(),
    "labels": test_df["relevance"].tolist()
}

In [23]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

# Применяем токенизацию
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Фильтруем ненужные колонки и переименовываем labels -> label (или наоборот)
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])
# Теперь в каждом датасете есть "input_ids", "attention_mask", (возможно "token_type_ids") и "labels".

Map:   0%|          | 0/49968 [00:00<?, ? examples/s]

Map:   0%|          | 0/4886 [00:00<?, ? examples/s]

Map:   0%|          | 0/5429 [00:00<?, ? examples/s]

### Настройка Trainer

In [24]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="binary")

    return {
        "accuracy": acc["accuracy"],
        "f1": f1["f1"]
    }

In [28]:
batch_size = 16  # Можно увеличить, если хватает GPU (16/32).
num_epochs = 3  # Чаще всего 3-5 эпох хватает для fine-tuning
learning_rate = 1e-5

training_args = TrainingArguments(
    output_dir="my_sbert_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Следим за f1
    greater_is_better=True,
    save_total_limit=2,
    logging_steps=100,               # Логировать каждые 100 шагов
    logging_dir="./logs",            # Директория для логов (если нужна)
    report_to="none",                # Чтобы логи шли просто в консоль
    disable_tqdm=False               # Разрешить tqdm-прогресс‐бар
    # FP16 ?
    # fp16=True,  # Если GPU поддерживает half precision, можно ускорить
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,  # Чтобы Trainer мог делать pad/collate
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

<ipython-input-29-e6fa9a6f883a>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.328900,0.332067,0.876586,0.608696
2,0.198000,0.465619,0.863078,0.591824


TrainOutput(global_step=6246, training_loss=0.2595382132211099, metrics={'train_runtime': 4683.7225, 'train_samples_per_second': 32.005, 'train_steps_per_second': 2.0, 'total_flos': 1.314713321422848e+16, 'train_loss': 0.2595382132211099, 'epoch': 2.0})

Я изменил batch_size с 8 на 16

In [31]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/my_sbert_model/final_2')
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/my_sbert_model/final_2')


('/content/drive/MyDrive/Colab Notebooks/my_sbert_model/final_2/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/my_sbert_model/final_2/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/my_sbert_model/final_2/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/my_sbert_model/final_2/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/my_sbert_model/final_2/tokenizer.json')

In [32]:
import numpy as np
from sklearn.metrics import recall_score

# Предположим, val_dataset (или test_dataset) — ваши данные для подбора порога
val_result = trainer.predict(val_dataset)
logits = val_result.predictions  # shape (num_samples, 2) для бинарной классификации
labels = val_result.label_ids    # истина (0 или 1)

# Превращаем логиты в вероятности для класса "True" (индекс 1)
import torch
probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
probs_true = probs[:, 1]  # вероятность принадлежности к классу True


In [34]:
thresholds = np.linspace(0, 1, 101)  # 0.00, 0.01, 0.02, ..., 1.00

best_threshold = None
best_diff = float('inf')  # чтобы минимизировать |recall_false - 0.40|
best_recall_false = None
best_recall_true = None

for thr in thresholds:
    # Бинарные предсказания по выбранному порогу
    pred = (probs_true >= thr).astype(int)  # 1, если prob >= thr

    # Считаем recall для каждого класса
    recs = recall_score(labels, pred, average=None)
    recall_false = recs[0]  # для класса 0
    recall_true  = recs[1]  # для класса 1

    # Проверяем условие recall_true >= 0.97
    if recall_true >= 0.95:
        # Смотрим, насколько recall_false близок к 0.40
        diff = abs(recall_false - 0.50)
        if diff < best_diff:
            best_diff = diff
            best_threshold = thr
            best_recall_false = recall_false
            best_recall_true = recall_true

# Проверяем, нашли ли мы подходящий порог
if best_threshold is not None:
    print(f"Подходящий порог: {best_threshold:.2f}")
    print(f"Recall False = {best_recall_false:.3f} (хотели около 0.40)")
    print(f"Recall True  = {best_recall_true:.3f} (>= 0.97)")
    print(f"Разница от 0.40 по False: {best_diff:.3f}")
else:
    print("Не удалось найти порог, при котором recall_true >= 0.97 и recall_false ~ 0.40.")


Подходящий порог: 0.02
Recall False = 0.272 (хотели около 0.40)
Recall True  = 0.979 (>= 0.97)
Разница от 0.40 по False: 0.228


### Проверка порога для тестовой выборки

In [35]:
import numpy as np
from sklearn.metrics import recall_score

# Предположим, val_dataset (или test_dataset) — ваши данные для подбора порога
test_result = trainer.predict(test_dataset)
logits = test_result.predictions  # shape (num_samples, 2) для бинарной классификации
labels = test_result.label_ids    # истина (0 или 1)

# Превращаем логиты в вероятности для класса "True" (индекс 1)
import torch
probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
probs_true = probs[:, 1]  # вероятность принадлежности к классу True


In [36]:
thresholds = np.linspace(0, 1, 101)  # 0.00, 0.01, 0.02, ..., 1.00

best_threshold = None
best_diff = float('inf')  # чтобы минимизировать |recall_false - 0.40|
best_recall_false = None
best_recall_true = None

for thr in thresholds:
    # Бинарные предсказания по выбранному порогу
    pred = (probs_true >= thr).astype(int)  # 1, если prob >= thr

    # Считаем recall для каждого класса
    recs = recall_score(labels, pred, average=None)
    recall_false = recs[0]  # для класса 0
    recall_true  = recs[1]  # для класса 1

    # Проверяем условие recall_true >= 0.97
    if recall_true >= 0.95:
        # Смотрим, насколько recall_false близок к 0.40
        diff = abs(recall_false - 0.50)
        if diff < best_diff:
            best_diff = diff
            best_threshold = thr
            best_recall_false = recall_false
            best_recall_true = recall_true

# Проверяем, нашли ли мы подходящий порог
if best_threshold is not None:
    print(f"Подходящий порог: {best_threshold:.2f}")
    print(f"Recall False = {best_recall_false:.3f} (хотели около 0.40)")
    print(f"Recall True  = {best_recall_true:.3f} (>= 0.97)")
    print(f"Разница от 0.40 по False: {best_diff:.3f}")
else:
    print("Не удалось найти порог, при котором recall_true >= 0.97 и recall_false ~ 0.40.")


Подходящий порог: 0.02
Recall False = 0.275 (хотели около 0.40)
Recall True  = 0.973 (>= 0.97)
Разница от 0.40 по False: 0.225
